# **Import and install**

In [1]:
import pandas as pd
import string
import emoji
import pickle
import torch.nn as nn
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from simpletransformers.ner import NERModel, NERArgs

# **Function**

In [2]:
ENGLISH_STOPWORDS = stopwords.words('english')
PUNCTUATION = string.punctuation
LEMMATIZER = WordNetLemmatizer()

In [3]:
def to_iob_tag(data_1, data_2, filename):
	arr = []
	for i, (cleaned_review, tag) in enumerate(zip(data_1, data_2)):
		review_tokenize = word_tokenize(cleaned_review)
		tag_tokenize = word_tokenize(tag)

		for (review_edit, tag_edit) in zip(review_tokenize, tag_tokenize):
			flag = f'Sentence {i + 1}'
			arr.append((flag, review_edit, tag_edit))

	tokenize_df = pd.DataFrame(arr)
	tokenize_df.rename(columns={0: 'Sentence #', 1: 'Word', 2: 'Tag'}, inplace=True)
	tokenize_df["Sentence #"] = LabelEncoder().fit_transform(tokenize_df["Sentence #"])
	tokenize_df.rename(columns={"Sentence #": "sentence_id", "Word": "words", "Tag": "labels"}, inplace =True)
	tokenize_df["labels"] = tokenize_df["labels"].str.upper()
	tokenize_df.dropna(inplace=True)
	tokenize_df.to_csv(f'datasets/{filename}.csv', index=False)

In [4]:
def clean_text(text):
  # Lowercase the text
  text = text.lower()
  text = emoji.replace_emoji(text, replace='')

  words = word_tokenize(text)
  words = [word for word in words if word not in ENGLISH_STOPWORDS]
  words = [LEMMATIZER.lemmatize(word) for word in words]
  words = [word for word in words if word not in PUNCTUATION]
  words = [word for word in words if word.isalnum()]

  # Join the cleaned tokens back into a string
  clean_text = ' '.join(words)
  return clean_text

# **Data Preparations**

## **Cleaning Data**

In [5]:
data = pd.read_csv('datasets/raw_datasets/amazon_one_plus_reviews.csv', encoding='latin1')
data.head()

,product,product_company,profile_name,review_title,review_rating,review_text,helpful_count,total_comments,review_country,reviewed_at,url,crawled_at,_id,verified_purchase,color,style_name,size_name,category,sub_category,images
0,"OnePlus Nord 5G (Gray Onyx, 8GB RAM, 128GB Sto...",OnePlus,Nikhil,*Read before you buy!!*,5.0 out of 5 stars,"\n Yea..pre-ordered on 28 July, got it on 4 A...",721 people found this helpful,3,India,4 August 2020,https://www.amazon.in/product-reviews/B08695ZS...,1.603020e+12,45ca015a-2e39-5650-a174-ba966dd1e51f,Verified Purchase,Marble Blue,8GB RAM + 128GB Storage,na,electronics,mobiles,['https://images-na.ssl-images-amazon.com/imag...
1,"OnePlus Nord 5G (Gray Onyx, 8GB RAM, 128GB Sto...",OnePlus,Amit,Near to mid range Perfection,5.0 out of 5 stars,"\n Got it delivered yesterday , used for abou...",436 people found this helpful,1,India,3 August 2020,https://www.amazon.in/product-reviews/B08695ZS...,1.603020e+12,6820f8ae-f3cd-5783-b826-5e5805376047,Verified Purchase,na,na,na,electronics,mobiles,[]
2,"OnePlus Nord 5G (Gray Onyx, 8GB RAM, 128GB Sto...",OnePlus,aishwarya,Great price!,5.0 out of 5 stars,\n An amazing phone!,322 people found this helpful,1,India,4 August 2020,https://www.amazon.in/product-reviews/B08695ZS...,1.603020e+12,bea91c43-8d65-5ef5-ab04-595a5f5cc542,Verified Purchase,na,na,na,electronics,mobiles,['https://images-na.ssl-images-amazon.com/imag...
3,"OnePlus Nord 5G (Gray Onyx, 8GB RAM, 128GB Sto...",OnePlus,vasu a.,Beast in OnePlus.,5.0 out of 5 stars,\n Brilliant..,243 people found this helpful,7,India,2 August 2020,https://www.amazon.in/product-reviews/B08695ZS...,1.603020e+12,76f68b53-5c52-56b2-9727-a5ab8c4bb4ff,Verified Purchase,na,na,na,electronics,mobiles,['https://images-na.ssl-images-amazon.com/imag...
4,"OnePlus Nord 5G (Gray Onyx, 8GB RAM, 128GB Sto...",OnePlus,Amazon Customer,Changed to Nord from 6t,5.0 out of 5 stars,\n I was skeptical about changing from One pl...,109 people found this helpful,2,India,4 August 2020,https://www.amazon.in/product-reviews/B08695ZS...,1.603020e+12,22228681-b884-56a1-b04e-87167f6f76e6,Verified Purchase,na,na,na,electronics,mobiles,[]


In [6]:
data = data['review_text']
data.head()

0    \n  Yea..pre-ordered on 28 July, got it on 4 A...
1    \n  Got it delivered yesterday , used for abou...
2                                \n  An amazing phone!
3                                      \n  Brilliant..
4    \n  I was skeptical about changing from One pl...
Name: review_text, dtype: object

In [7]:
data.dropna(inplace=True)

In [8]:
data.isna().any()

False

In [9]:
data = data.apply(clean_text)
data.head()

0    yea 28 july got 4 august package nice without ...
1    got delivered yesterday used hour tell first m...
2                                        amazing phone
3                                            brilliant
4    skeptical changing one plus 6t nord 845 still ...
Name: review_text, dtype: object

In [10]:
data.to_csv('datasets/cleaned/clean_amazon_one_plus_reviews.csv', index=False)

In [11]:
data = pd.read_csv('datasets/raw_datasets/apple_reviews.csv', encoding='latin1')
data.head()

,category,rating,product,helpful_count,crawled_at,sub_category,asin,review_country,raw_description,description,profile_name,reviewed_at,_id,average_rating,product_company,review_title,review_uniq_id,images,verified_purchase,total_comments
0,Mobiles & Accessories,1,New Apple iPhone 11 (64GB) - Purple,356,2021-01-26 13:54:03,Smartphones & Basic Mobiles,B08L8CHS2X,India,"<span class=""a-size-base review-text review-te...",\n\n The phone seems to be owned by some shiv...,Amazon Customer,2020-06-10 00:00:00,f7030d10-c987-536b-a356-f642cb1ec5e3,4.5,Apple,Second hand device,RCNKLUGAMJBPH,[],True,0
1,Mobiles & Accessories,1,New Apple iPhone 11 (64GB) - Purple,363,2021-01-26 13:54:03,Smartphones & Basic Mobiles,B08L8CHS2X,India,"<span class=""a-size-base review-text review-te...",\n\n Its my first iphone and now Iâm feelin...,Bhawna juyal,2019-10-10 00:00:00,775c9997-8c2c-5dda-9faf-0f0e29ac9cb0,4.5,Apple,Heating issue,R1HWMMIJ21WTZR,[],True,0
2,Mobiles & Accessories,1,New Apple iPhone 11 (64GB) - Purple,239,2021-01-26 13:54:03,Smartphones & Basic Mobiles,B08L8CHS2X,India,"<span class=""a-size-base review-text review-te...",\n\n I found this black spot while doing set ...,Vinayak,2020-06-06 00:00:00,089a10aa-ef92-577c-93f7-fd85e26f6bf0,4.5,Apple,Black spot found on the new phone.,R3FZWAU9I8B7Q5,['https://images-na.ssl-images-amazon.com/imag...,True,0
3,Mobiles & Accessories,1,New Apple iPhone 11 (64GB) - Purple,277,2021-01-26 13:54:03,Smartphones & Basic Mobiles,B08L8CHS2X,India,"<span class=""a-size-base review-text review-te...",\n\n Samsung s10 is much better than iphone 1...,Shipra Goyal,2019-10-23 00:00:00,217a69c8-f3d6-58b0-9ec0-8a1e3514e24e,4.5,Apple,Samsung s10 is better than iphone,R386FDLMAZDUPG,[],True,0
4,Mobiles & Accessories,1,New Apple iPhone 11 (64GB) - Purple,191,2021-01-26 13:54:03,Smartphones & Basic Mobiles,B08L8CHS2X,India,"<span class=""a-size-base review-text review-te...",\n\n hiI had purchased this phone in first we...,gaurav garg,2020-06-24 00:00:00,5e2e9ea9-a32d-51fe-bd7b-f0ebe5749ebd,4.5,Apple,Defective model,RXAOS1IG04N09,[],True,0


In [12]:
data = data['description']
data.head()

0    \n\n  The phone seems to be owned by some shiv...
1    \n\n  Its my first iphone and now Iâm feelin...
2    \n\n  I found this black spot while doing set ...
3    \n\n  Samsung s10 is much better than iphone 1...
4    \n\n  hiI had purchased this phone in first we...
Name: description, dtype: object

In [13]:
data.dropna(inplace=True)

In [14]:
data.isna().any()

False

In [15]:
data = data.apply(clean_text)
data.head()

0    phone seems owned shivang gupta earlier detail...
1    first iphone feeling like wasted money coz hea...
2    found black spot set given defective piece abl...
3                    samsung s10 much better iphone 11
4    hii purchased phone first week june 2020 trust...
Name: description, dtype: object

In [16]:
data.to_csv('datasets/cleaned/clean_apple_reviews.csv', index=False)

## **Merge Dataset**

In [17]:
review_1 = pd.read_csv('datasets/cleaned/clean_amazon_one_plus_reviews.csv', encoding='latin1')
review_2 = pd.read_csv('datasets/cleaned/clean_apple_reviews.csv', encoding='latin1')

In [18]:
review_1.head()

,review_text
0,yea 28 july got 4 august package nice without ...
1,got delivered yesterday used hour tell first m...
2,amazing phone
3,brilliant
4,skeptical changing one plus 6t nord 845 still ...


In [19]:
review_2.head()

,description
0,phone seems owned shivang gupta earlier detail...
1,first iphone feeling like wasted money coz hea...
2,found black spot set given defective piece abl...
3,samsung s10 much better iphone 11
4,hii purchased phone first week june 2020 trust...


In [20]:
review_1.rename(columns={'review_text': 'review'}, inplace=True)
review_2.rename(columns={'description': 'review'}, inplace=True)

In [21]:
review = [review_1, review_2]
review = pd.concat(review)
review.head()

,review
0,yea 28 july got 4 august package nice without ...
1,got delivered yesterday used hour tell first m...
2,amazing phone
3,brilliant
4,skeptical changing one plus 6t nord 845 still ...


In [22]:
review.tail()

,review
19673,awsum phone said product deliver 1 day pay 1 d...
19674,purchased deal great phone fast wonderful pict...
19675,user friendly
19676,iphone 7 7 plus deeply unusual device full agg...
19677,using device 7 month ben awesome love fluidic ...


In [23]:
review.dropna(inplace=True)

In [24]:
review = review.sample(frac=1)
review.head()

,review
17130,gud product price
1515,one best phone far price range
27910,mobail nice betrry bad fast charging lowest ca...
9451,battery worrest
28972,bought phone horrible experience redmi note 8 ...


In [25]:
review.to_csv('datasets/cleaned/merged_reviews.csv', index=False)

## **Datasets for CPU Name**

In [26]:
data = pd.DataFrame({'cpu_name': ['A10 Fusion',
'A11 Bionic',
'A12 Bionic',
'A13 Bionic',
'A14 Bionic',
'A15 Bionic',
'A16 Bionic',
'A17 Pro',
'Apple A9',
'Dimensity 1000 Plus',
'Dimensity 1050',
'Dimensity 1080',
'Dimensity 1100',
'Dimensity 1200',
'Dimensity 1300',
'Dimensity 6020',
'Dimensity 6080',
'Dimensity 6100 Plus',
'Dimensity 6300',
'Dimensity 700',
'Dimensity 7020',
'Dimensity 7025',
'Dimensity 7030',
'Dimensity 7050',
'Dimensity 720',
'Dimensity 7200',
'Dimensity 7200 Ultra',
'Dimensity 800',
'Dimensity 800U',
'Dimensity 8020',
'Dimensity 8050',
'Dimensity 810',
'Dimensity 8100',
'Dimensity 820',
'Dimensity 8200',
'Dimensity 8300',
'Dimensity 900',
'Dimensity 9000',
'Dimensity 9000 Plus',
'Dimensity 920',
'Dimensity 9200',
'Dimensity 9200 Plus',
'Dimensity 930',
'Dimensity 9300',
'Exynos 1280',
'Exynos 1330',
'Exynos 1380',
'Exynos 1480',
'Exynos 2100',
'Exynos 2200',
'Exynos 2400',
'Exynos 7870',
'Exynos 7884B',
'Exynos 7885',
'Exynos 7904',
'Exynos 850',
'Exynos 8890',
'Exynos 8895',
'Exynos 9610',
'Exynos 9611',
'Exynos 980',
'Exynos 9810',
'Exynos 9820',
'Exynos 9825',
'Exynos 990',
'Google Tensor',
'Helio A22',
'Helio G25',
'Helio G35',
'Helio G36',
'Helio G37',
'Helio G70',
'Helio G80',
'Helio G85',
'Helio G88',
'Helio G90T',
'Helio G95',
'Helio G96',
'Helio G99',
'Helio P22',
'Helio P35',
'Helio P60',
'Helio P65',
'Helio P70',
'Helio P90',
'Helio P95',
'Kirin 659',
'Kirin 710',
'Kirin 710A',
'Kirin 710F',
'Kirin 810',
'Kirin 9000',
'Kirin 9000S',
'Kirin 9010',
'Kirin 960',
'Kirin 970',
'Kirin 980',
'Kirin 990 (5G)',
'MediaTek MT6737',
'Snapdragon 4 Gen 1',
'Snapdragon 4 Gen 2',
'Snapdragon 430',
'Snapdragon 435',
'Snapdragon 439',
'Snapdragon 450',
'Snapdragon 460',
'Snapdragon 480',
'Snapdragon 480 Plus',
'Snapdragon 6 Gen 1',
'Snapdragon 625',
'Snapdragon 630',
'Snapdragon 632',
'Snapdragon 636',
'Snapdragon 660',
'Snapdragon 662',
'Snapdragon 665',
'Snapdragon 670',
'Snapdragon 675',
'Snapdragon 678',
'Snapdragon 680',
'Snapdragon 685',
'Snapdragon 690',
'Snapdragon 695',
'Snapdragon 7 Gen 1',
'Snapdragon 7 Gen 3',
'Snapdragon 7 Plus Gen 2',
'Snapdragon 7 Plus Gen 3',
'Snapdragon 710',
'Snapdragon 712',
'Snapdragon 720G',
'Snapdragon 730',
'Snapdragon 730G',
'Snapdragon 732G',
'Snapdragon 750G',
'Snapdragon 765G',
'Snapdragon 778G',
'Snapdragon 778G Plus',
'Snapdragon 780G',
'Snapdragon 782G',
'Snapdragon 7s Gen 2',
'Snapdragon 8 Gen 1',
'Snapdragon 8 Gen 2',
'Snapdragon 8 Gen 3',
'Snapdragon 8 Plus Gen 1',
'Snapdragon 820',
'Snapdragon 821',
'Snapdragon 835',
'Snapdragon 845',
'Snapdragon 855',
'Snapdragon 855 Plus',
'Snapdragon 860',
'Snapdragon 865',
'Snapdragon 865 Plus',
'Snapdragon 870',
'Snapdragon 888',
'Snapdragon 888 Plus',
'Snapdragon 8s Gen 3',
'Tensor G2',
'Tensor G3',
'Tiger T610',
'Tiger T612',
'Tiger T616',
'Tiger T618',
'Tiger T700',
'Unisoc SC9863A',
'Unisoc T606',
'Unisoc T820']})

In [27]:
data.to_csv('datasets/raw_datasets/more_cpu_name.csv', index=False)

## **CPU Name**

In [28]:
cpu_1 = pd.read_csv('datasets/raw_datasets/cpu_name.csv', encoding='latin1')
cpu_2 = pd.read_csv('datasets/raw_datasets/more_cpu_name.csv', encoding='latin1')

In [29]:
cpu_1 = pd.DataFrame({'cpu_name': cpu_1['cpuName']})
cpu_2 = pd.DataFrame({'cpu_name': cpu_2['cpu_name']})

In [30]:
cpu_name = pd.concat([cpu_1, cpu_2])
cpu_name.head()

,cpu_name
0,A15 Bionic
1,A15 Bionic
2,A15 Bionic
3,M1
4,M1


In [31]:
cpu_name.dropna(inplace=True)
len(cpu_name)

355

In [32]:
cpu_name = cpu_name.drop_duplicates()
len(cpu_name)

230

In [33]:
cpu_name = cpu_name.drop(cpu_name.index[[71]])
cpu_name

,cpu_name
0,A15 Bionic
3,M1
7,A14 Bionic
12,Tensor
14,A13 Bionic
...,...
162,Tiger T618
163,Tiger T700
164,Unisoc SC9863A
165,Unisoc T606


In [34]:
def tagging_cpu_data(text):
   text_dict = ''
   word_token = word_tokenize(text)
   
   for i, word in enumerate(word_token):
      if i == 0:
         text_dict = text_dict + 'B-performance '
      else:
         text_dict = text_dict + 'I-performance '
      
   return text_dict

In [35]:
cpu_name['tag'] = cpu_name['cpu_name'].apply(tagging_cpu_data)
cpu_name = cpu_name.sample(frac=1)
cpu_name.head()

,cpu_name,tag
150,Snapdragon 860,B-performance I-performance
14,A13 Bionic,B-performance I-performance
165,Exynos 9611,B-performance I-performance
93,Kirin 9000,B-performance I-performance
33,Dimensity 820,B-performance I-performance


In [36]:
cpu_name['cpu_name'] = cpu_name['cpu_name'].apply(clean_text)
cpu_name.head()

,cpu_name,tag
150,snapdragon 860,B-performance I-performance
14,a13 bionic,B-performance I-performance
165,exynos 9611,B-performance I-performance
93,kirin 9000,B-performance I-performance
33,dimensity 820,B-performance I-performance


In [37]:
cpu_name.to_csv('datasets/tagged/cpu_name.csv', index=False)

In [38]:
to_iob_tag(cpu_name['cpu_name'], cpu_name['tag'], 'cpu-iob-ner')

## **Tagging Data**

In [39]:
import re

In [40]:
data = pd.read_csv('datasets/cleaned/merged_reviews.csv', encoding='latin1')
data.head()

,review
0,gud product price
1,one best phone far price range
2,mobail nice betrry bad fast charging lowest ca...
3,battery worrest
4,bought phone horrible experience redmi note 8 ...


In [41]:
cpu_name = pd.read_csv('datasets/cpu-iob-ner.csv', encoding='latin1')
cpu_name.head()

,sentence_id,words,labels
0,0,snapdragon,B-PERFORMANCE
1,0,860,I-PERFORMANCE
2,111,a13,B-PERFORMANCE
3,111,bionic,I-PERFORMANCE
4,152,exynos,B-PERFORMANCE


In [45]:
cpu_b_tag = cpu_name.loc[cpu_name['labels'] == 'B-PERFORMANCE']
cpu_i_tag = cpu_name.loc[cpu_name['labels'] == 'I-PERFORMANCE']

cpu_b_name_word = set(cpu_b_tag['words'].tolist())
cpu_b_name_word = list(cpu_b_name_word)
cpu_b_name_word = [i.lower() for i in cpu_b_name_word]
cpu_b_name_word.remove('google')
						 
cpu_i_name_word = set(cpu_i_tag['words'].tolist())
cpu_i_name_word = list(cpu_i_name_word)
cpu_i_name_word = [i.lower() for i in cpu_i_name_word]
cpu_i_name_word.remove('tensor')

In [46]:
camera = ['image', 'picture', 'camera', 'video', 'photography', 'ois', 'eis', 'stabilization']
performance = ['faster', 'cpu', 'performance', 'lag', 'processor', 'power', 'smooth', 'os', 'specification', 'android', 'ios']
screen = ['screen', 'display', 'nits', 'brightness', 'ips', 'hz', 'hertz', 'lcd']
charging = ['charging', 'charge', 'watt', 'port', 'usb']
audio = ['mic', 'microphone', 'audio', 'speaker', 'volume', 'loudspeaker', 'headphone']
connection = ['connection', 'wifi', 'bluetooth']
storage = ['ram', 'rom', 'storage', 'emmc', 'ufs', 'mbps', 'gbps', 'microsd']
feature = ['feature', 'sensor', 'sensors', 'hdr']

In [47]:
def tagging_data(text):
  word_token = word_tokenize(text)
  text_dict = ['O'] * len(word_token)

  for i, word in enumerate(word_token):
    if text_dict[i] != 'O':
      continue
    if word in camera:
      text_dict[i] = 'B-camera'
    elif word in performance:
      text_dict[i] = 'B-performance'
    elif word in screen:
      text_dict[i] = 'B-screen'
    elif word in charging:
      text_dict[i] = 'B-charging'
    elif word in audio:
      if re.match(r'\b(audio|headphone)\b', word) and i < len(word_token) - 1 and word_token[i + 1] == 'jack':
        text_dict[i] = 'B-audio'
        text_dict[i + 1] = 'I-audio'
      else:
        text_dict[i] = 'B-audio'
    elif word in connection:
      text_dict[i] = 'B-connection'
    elif word in storage:
      text_dict[i] = 'B-storage'
    elif word in feature:
      text_dict[i] = 'B-feature'
    elif word in cpu_b_name_word:
      text_dict[i] = 'B-performance'
    elif word in cpu_i_name_word and (text_dict[i - 1] == 'B-performance' or text_dict[i - 1] == 'I-performance'):
      text_dict[i] = 'I-performance'
    else:
      if re.match(r'\b\d+(?:gb|mb|kb)\b', word):
        text_dict[i] = 'B-storage'
        if i < len(word_token) - 1 and re.match(r'\b(ram|rom)\b', word_token[i + 1]):
          text_dict[i + 1] = 'I-storage'
      elif re.match(r'\b(amoled|oled)\b', word):
        text_dict[i] = 'B-screen'
        if i < len(word_token) - 1 and re.match(r'\b(display|screen)\b', word_token[i + 1]):
          text_dict[i + 1] = 'I-screen'
      elif re.match(r'\b\d+\-*(?:mp|megapixel)\b', word):
        text_dict[i] = 'B-camera'
      elif re.match(r'\b\d+(?:hz|hertz|p)\b', word):
        text_dict[i] = 'B-screen'
        if i < len(word_token) - 1 and re.match(r'\b(resolution|refresh)\b', word_token[i + 1]):
          text_dict[i + 1] = 'I-screen'
          if i + 2 < len(word_token) - 1 and re.match(r'\brate\b', word_token[i + 2]):
            text_dict[i + 2] = 'I-screen'
      elif word.isnumeric() and i < len(word_token) - 1 and re.match(r'\b(gb|mb|kb)\b', word_token[i + 1]):
        text_dict[i] = 'B-storage'
        text_dict[i + 1] = 'I-storage'
        if i + 2 < len(word_token) - 1 and re.match(r'\b(ram|rom)\b', word_token[i + 2]):
          text_dict[i + 2] = 'I-storage'
      elif word.isnumeric() and i < len(word_token) - 1 and re.match(r'\b(mp|megapixels?)\b', word_token[i + 1]):
        text_dict[i] = 'B-camera'
        text_dict[i + 1] = 'I-camera'
        if i + 2 < len(word_token) - 1 and re.match(r'\b(systems?|sensors?)\b', word_token[i + 2]):
          text_dict[i + 2] = 'I-camera'
      elif word == 'fingerprint' and i < len(word_token) - 1 and re.match(r'\b(scanner|reader)\b', word_token[i + 1]):
        text_dict[i] = 'B-feature'
        text_dict[i + 1] = 'I-feature'
      elif word == 'face' and i < len(word_token) - 1 and re.match(r'\b(unlock|id)\b', word_token[i + 1]):
        text_dict[i] = 'B-feature'
        text_dict[i + 1] = 'I-feature'
      elif word == 'sd' and i < len(word_token) - 1 and word_token[i + 1] == 'card':
        text_dict[i] = 'B-storage'
        text_dict[i + 1] = 'I-storage'
      elif word == 'micro' and i < len(word_token) - 1 and word_token[i + 1] == 'sd':
        text_dict[i] = 'B-storage'
        text_dict[i + 1] = 'I-storage'
      elif re.match(r'\b(macro|main|front|ultrawide|wide|zoom|rear|selfie|depth|telephoto)\b', word) and i < len(word_token) - 1 and re.match(r'\bcameras?\b', word_token[i + 1]):
        text_dict[i] = 'B-camera'
        text_dict[i + 1] = 'I-camera'
        if i + 2 < len(word_token) - 1 and re.match(r'\b(systems?|sensors?)\b', word_token[i + 2]):
          text_dict[i + 2] = 'I-camera'
      elif i < len(word_token) - 1 and re.match(r'\b(cameras?)\b', word) and re.match(r'\bsensors?\b', word_token[i + 1]):
        text_dict[i] = 'B-camera'
        text_dict[i + 1] = 'I-camera'
      elif word == 'operating' and i < len(word_token) - 1 and word_token[i + 1] == 'system':
        text_dict[i] = 'B-performance'
        text_dict[i + 1] = 'I-performance'
      elif re.match(r'\b(proximity|accelerometer|gyroscope|barometer|biometrics)\b', word):
        text_dict[i] = 'B-feature'
        if i < len(word_token) - 1 and re.match(r'\bsensors?\b', word_token[i + 1]):
          text_dict[i + 1] = 'I-feature'
      elif re.match(r'\b(augmented|virtual)\b', word) and i < len(word_token) - 1 and word_token[i + 1] == 'reality':
        text_dict[i] = 'B-feature'
        text_dict[i + 1] = 'I-feature'
      elif word == 'battery': 
        text_dict[i] = 'B-battery'
        if i < len(word_token) - 1 and re.match(r'\b(capacity|life)\b', word_token[i + 1]):
          text_dict[i + 1] = 'I-battery'
      elif re.match(r'\b\d+mah\b', word):
        text_dict[i] = 'B-battery'
      elif word.isnumeric() and i < len(word_token) - 1 and word_token[i + 1] == 'mah':
        text_dict[i] = 'B-battery'
        text_dict[i + 1] = 'I-battery'
      elif word == 'button':
        text_dict[i] = 'B-button'
        if re.match(r'\b(power|volume|lock|mute|home|back)\b', word_token[i - 1]):
          text_dict[i] = 'I-button'
          text_dict[i - 1] = 'B-button'
      elif word == 'connectivity':
        text_dict[i - 1] = 'B-connection'
        text_dict[i] = 'I-connection'
      elif re.match(r'\b\dg\b', word):
        text_dict[i] = 'B-connection'
        if i < len(word_token) - 1 and re.match(r'\b(connectivity|network)\b', word_token[i + 1]):
          text_dict[i + 1] = 'I-connection'
      elif word == 'network':
        text_dict[i] = 'B-connection'
        if word_token[i - 1] == 'mobile':
          text_dict[i - 1] = 'B-connection'
          text_dict[i] = 'I-connection'
      elif re.match(r'\b(f|q)?hd\b', word):
        text_dict[i] = 'B-screen'
        if i < len(word_token) - 1 and word_token[i + 1] == 'display':
          text_dict[i + 1] = 'I-screen'
      elif re.match(r'\b\d+\-*ppi\b', word):
        text_dict[i] = 'B-screen'
      elif word.isnumeric() and i < len(word_token) - 1 and word_token[i + 1] == 'ppi':
        text_dict[i] = 'B-screen'
        text_dict[i + 1] = 'I-screen'
      elif re.match(r'\bip6\d\b', word):
        text_dict[i] = 'B-feature'
        if i < len(word_token) - 1 and word_token[i + 1] == 'rating':
          text_dict[i + 1] = 'I-feature'
      elif re.match(r'\b\d+(?:watt|w)\b', word):
        text_dict[i] = 'B-charging'
        if i < len(word_token) - 1 and re.match(r'\b(charge|charging|charger)\b', word_token[i + 1]):
          text_dict[i + 1] = 'I-charging'
      elif word.isnumeric() and i < len(word_token) - 1 and re.match(r'\b(watt|w)\b', word_token[i + 1]):
        text_dict[i] = 'B-charging'
        text_dict[i + 1] = 'I-charging'
      elif re.match(r'\b(fast|quick)\b', word) and i < len(word_token) - 1 and re.match(r'\b(charge|charging|charger)\b', word_token[i + 1]):
        text_dict[i] = 'B-charging'
        text_dict[i + 1] = 'I-charging'

  tokenized_text = ' '.join(text_dict)
  return tokenized_text

In [48]:
data['tag'] = data['review'].apply(tagging_data)
data.head()

,review,tag
0,gud product price,O O O
1,one best phone far price range,O O O O O O
2,mobail nice betrry bad fast charging lowest ca...,O O O O B-charging I-charging O B-camera O O O...
3,battery worrest,B-battery O
4,bought phone horrible experience redmi note 8 ...,O O O O O O O O O O O O O


In [49]:
data.to_csv('datasets/tagged/reviews.csv', index=False)

In [50]:
# Data from GSMArena
gsm_arena_review = [
	'''Huawei Pura 70 Ultra is the headliner of the reimagined Photography series by Huawei. It packs the best camera the Chinese maker could conjure up, coupled with the best design, screen, Kirin chipset, and battery technology, of course.''',
	'''Despite the name change, the Huawei P series is alive and kicking, promising as ever, even without Leica or other trendy partnership being present for marketing hype. And, yes, the US sanctions are still at play, but Huawei seems to be on the verge of making those irrelevant.''',
	'''Back to what we do best - smartphone inspection - we are nothing short of thrilled by our time spent with the Huawei Pura 70 Ultra. And while we are in the process of reviewing it, we decided to give you a glimpse of what's coming your way.''',
	'''The Pura 70 Ultra is one of the few phones on the market you can immediately pick among a hundred phones in a second. The triangular camera design and unique back panel are quite nice, even if the housing is, well, huge.''',
	'''Then we have a large 6.8-inch LTPO OLED screen of 1260p resolution, 1B colors and 120Hz refresh rate. The Kirin 9010 chipset with 5G connectivity will probably initiate a new portion of investigations, but it does look promising with improved capabilities.''',
	'''The camera is the real showstopper here and rightfully, so. The retractable main camera is thoroughly interesting with its unique mechanical design, sure, but we cannot wait to test its 1" sensor, one that makes use of sensor-shift stabilization, variable aperture lens and laser-assisted AF. Next is a 50MP telephoto camera for 3.5x optical zoom and up to 10x hybrid zoom, one that can do macro photos from as close as 5cm away! The 40MP ultrawide camera promises really wide field of view with its 13mm aperture lens, and it is AF-capable.''',
	'''The 13MP selfie camera also has wide-angle lens supports autofocus, too.''',
	'''Finally, the Huawei Pura 70 Ultra packs a 5200mAh battery capable of 100W wired and 80W wireless charging, plus 20W reverse wireless charging!''',
	'''The Pura 70 Ultra ships into one large and thick luxury-looking box. Inside you will find the phone itself, one premium case with a matching color but a different P-styled texture, a 100W power adapter and a USB cable.''',
	'''Notice the charger has both USB-C and USB-A ports, though it is not a dual-charger - you can use one of those at a time.''',
	'''The Honor and Porsche Design design collaboration continues and after the Magic V2 RSR we now have the Magic6 RSR. This one is a reimagined Magic6 Pro that  comes with a couple of hard to spot hardware improvements - in contrast to the V2 situation where the differences were skin-deep.''',
	'''The Porsche Design Magic6 RSR builds on the regular model's camera system by adding some extra autofocusing assistance. It comes in the form of a 1200-point LiDAR matrix that enables focusing speeds at up to 60fps. Great for pet photo shoots then. ''',
	'''Honor also says it's using a different and more advanced version of the primary camera's sensor that pushes its dynamic range capability up to 15EV. The weather contributed with clouds for that test.''',
	'''The display is also different on the Magic6 RSR, compared to the Magic6 Pro. The size and resolution are unchanged, but it's now a dual-layer OLED panel - or Tandem display - promising improved longevity and a nudge in max brightness (not that the non-RSR version was lacking in that respect).''',
	'''We'll have a look at the Porsche Design Honor Magic6 RSR and compare it side by side with the regular version, focusing on the differences mentioned - sort of a mini-review of the display and its effect on battery life, and a brief exploration into the main camera and how it stacks up next to that of the Magic6 Pro.''',
	'''The presentation of the Porsche Design version is predictably more upscale than that of the regular Magic6 Pro. The contrast is now even more stark than it would have been in previous years, with the Pro switching to the phone-only approach in 2024 - not so with the RSR.''',
	'''The Porsche Design Magic6 ships in an oversized graphite colored box that does contain more than you'd normally expect. The most surprising development (if you haven't already seen the Magic V2 RSR unboxing) is the inclusion of not one, but two, separate chargers - one with a UK style plug (Type G) and another with a Euro plug (Type F). There's also a separate cable for each adapter. Maybe all the chargers saved from mainstream Magics have been diverted to RSR boxes.''',
	'''The package also includes a snap-on protective case for the back of the phone. It's covered in a leather-like material on the outside, which looks just like that on the Magic V2 RSR's case, but is somehow ever so slightly less supple. It's still pretty nice, of course, it's just that the other one is nicer. On the inside, the alcantara or similar material has a fine pattern of hexagons, just like the phone's camera island.'''
	'''It's been a while now since we originally saw the Honor Magic V2 and got our initial hands-on experience at the Chinese announcement event. Since then, the phone finally had its global launch alongside an RSR Porsche Design edition. It's almost May now, and we finally we finally get an opportunity to review both devices. We are only pointing out the timeframe of things since, in today's dynamic market, all that time is more than enough to make a phone not necessarily irrelevant but definitely less exciting.</p>''',
	'''But not the Honor Magic V2, though. It remains special, like really special, for both foldable fans and the smartphone realm in general. Sure, it might be running a generation-old Snapdragon 8 Gen 2 chipset at this point, but that's about the only non-current and non-revolutionary thing about the Magic V2.</p>''',
	'''Let's start with what is probably the most impressive aspect of the Magic V2 - its size and weight. Of course, in absolute terms, it is far from compact, measuring 156.7x74.1x9.9mm folded and tipping the scale at 231g (with a few asterisks). It may not sound <i>that</i> impressive until you realize that Honor has made the first foldable in this form factor with a less than 10mm thickness.</p>''',
	'''And if that's not impressive enough, consider that the non-folding Samsung Galaxy S24 Ultra measures 162.3 x 79 x 8.6 mm and weighs in at 232 grams. With the V2, you can now get a whole extra display (and a large 7.92-inch one at that) and no other major compromises, including a hefty 5,000 mAh battery a potent triple camera setup and even an extra selfie camera all in roughly the same form factor as Samsung's or Apple's current flagship models. Foldables really have come a long way, and despite its relative age, the Honor Magic V2 represents the current pinnacle of this form factor.</p>''',
	'''It almost feels like Honor broke some laws of physics with the Magic V2. A lot of the space and weight savings come from the new silicon-carbon battery used in the Magic V2. It boasts impressive energy density, making it super thin, which explains how you can get that capacity inside a svelte body like the Magic V2's.</p>''',
	'''There are many more refinements and upgrades in the Magic V2, which is technically the third major iteration in Honor's foldable line. We'll definitely go over everything in detail, but some highlights include the 120Hz refresh rate on the foldable display and LTPO tech on both panels plus a brand new camera setup with OIS and 8x8 dToF Laser AF on the main cam.</p>''',
	'''We'll also mention that while we were trying to fit the V2 in the hectic review schedule, a package arrived at our doorstep with another V2, a v2 of the V2, so to speak. The Porsche Design Honor Magic V2 RSR is a much more exclusive repackaging of otherwise the same hardware, and we'll say a few words on that one as well.</p>''',
	'''Unlike most mainstream flagships, foldables still tend to present themselves as more premium products. Hence, you can usually expect more attention to the retail bundle like back in the day. This is definitely true for the Magic V2. It comes in a large and very posh-feeling box that provides plenty of protection during shipping.</p>''',
	'''The accessory package is quite lavish, especially in modern terms. You get one of Honor's 66W SuperCharge adapters in the box which pairs perfectly with the charging capabilities of the phone. Interestingly enough, the charger still uses a Type-A connection, which is why you get a Type-A to Type-C USB cable in the box. Also included is a snap-on case with a carbon fiber pattern as well as a kickstand.</p>''',
	'''And if you thought that was premium, wait till you see the Porsche Design version. An even nicer box is just the start, but the contents do go the extra mile.</p>''',
	'''For one, you get not one, but two chargers, one with a UK style plug (Type G) and another with a Euro plug (Type F) - because if you can afford the RSR version, you clearly are a globetrotter that can't be bothered with socket adapters. You also get two USB cables, because at this point, why not?</p>''',
	'''The bundle includes an even nicer back cover that's clad in what is either real leather or looks and feels very close to it, plus it has real thread stitching. ''',
	'''Both Magic V2s can be operated with a stylus, but only the Porsche Design version comes with one in the box. And it's a pretty fancy-looking one too, as befitting the brand, plus it comes with a carrying case of its own. Life of the rich and the famous, indeed.</p>''',
	'''Unintuitively, the Honor Pad 9 (a.k.a. the Tablet 9 in China) is has a large 12.1-inch screen. It covers all the basics with an eight-speaker audio setup and even stylus support, all for a fairly reasonable price of £299.99 or £349.99 with the optional Bluetooth keyboard attachment.'''
	'''The Pad 9 does not stray past the basics. There is no expandable storage, no 3.5mm audio jack, no support for positioning, and no NFC or a radio receiver. The camera setup is also fairly basic, with a 13MP snapper on the back and an 8MP one on the front.'''
	'''Then again, with the recent surge in "value-driving" devices in the tablet space, we have to assume that market research likely indicates that few users are looking for truly premium tablets, so the Honor Pad 9 might be well positioned in the midrange.'''
	'''The Honor Pad 9 ships in a basic two-piece cardboard box. There is no plastic in the package, which is always appreciated from an ecological standpoi'''
	'''The box cover has a large, to-scale image on the front of the tablet itself, along with the proud proclamation that it supports "the Google App and Google Chrome".'''
	'''There is plenty of space inside the box, and quite a few compartments for different accessories. Our unit only came with an Honor SuperCharge charger and a basic USB Type-A to Type-C cable. You don't get the keyboard or the stylus in the main box.'''
	'''We also got the optional Bluetooth keyboard in for review as well. It comes in an equally basic flip-up cardboard box with no plastics in sight. However, there is some foam to keep the keyboard from banging up against itself (in the case part).'''
	'''Unfortunately, we don't have the optional Honor Choice Pencil stylus to review at this time.''',
	'''Motorola is all about sensorial experiences this year and shifting the attention from specs to lifestyle - or at least that's the idea they're trying to sell us on with the Edge 50 Pro. Colors, materials, experiences, and AI are where the marketing focus is, but we'll try to remain pragmatic and not get too tangled in the esoteric.''',
	'''You can't miss the Pantone collaboration on the outside - as with previous models from the company, the Edge 50 Pro comes in at least one unusual hue that's been okayed by the color matching specialists; it's Luxe Lavender in this case. But that's been done before, and there's more Pantone now - the display and camera are Pantone-validated too, both of which are firsts in the industry (for whatever that's worth).''',
	'''More to our liking are numbers in general and the 6.7-inch OLED is both sharp (1220p) and fast (144Hz), not to mention bright (2000nits, Motorola says). There are specs to be appreciated in the camera field too - first and foremost, the f/1.4 aperture on the main camera. It's also nice to see a proper three-camera setup, telephoto included, and the fact that the ultrawide has autofocus deserves praise right from the outset.''',
	'''We're less enthusiastic about the heart of the Edge 50 Pro, the Snapdragon 7 Gen 3. Perhaps you recall that the Edge 40 Pro had the flagship Snapdragon last year, but this year's Pro is different - a restructuring of the lineup in 2024 means there's an Ultra at the top, like there was one generation prior, and that one gets the top-level chipset. The Pro, meanwhile, steps down to more of an upper-midrange status - we'll see where it stands on the value meter as we go along.''',
	'''The charging capability is about as flagship-grade as they come, though - at 125W and 18 minutes from empty to full, the specs are more than promising - albeit with some caveats. The 50W wireless charging rating is also a welcome improvement over the 15W of the previous generation, even if the 4,500mAh capacity isn't overly generous. We do like the relatively compact size and pocket-friendly weight, and the IP68 rating is most welcome too (also, befitting the 'Pro' moniker).''',
	'''Motorola unboxings have become somewhat of a treat in recent years, thanks to the company's practice of using fragrance on the inside of the box - that sure is 'a unique sensorial experience' that you won't be getting from other brands. We'll be quick to admit that there's little practical benefit to this, but that doesn't mean we like it any less. That's in addition to the packaging being plastic-free and made of 80% recycled materials, which is always nice.''',
	'''Inside, in addition to the phone, you'll be getting a TurboPower adapter. Its maximum output will vary depending on region and/or version of the phone. Our 12GB/512GB Euro spec includes the 125W charger, and that's what the 12GB/256GB option will also be getting, but the 8GB/128GB and 8GB/256GB variants will be bundled with a 68W unit (at least as best as we can tell - it's worth double checking with your vendor). A USB-C cable is included too.''',
	'''Also part of the package is a snap-on back cover with a semi-transparent frosted effect. The color of this accessory will be matched to the phone's colorway and for our Luxe Lavender review unit the hue is called Blue Heron (in Pantone speak again).''',
	'''The success of the Galaxy A54 has set the bar high for the just-released Galaxy A55. After the Samsung Galaxy S and Z foldable series, the Galaxy A5x series is likely the most popular in the company's portfolio for good reason. ''',
	'''This year's model makes gains in key areas. The new A55 boasts a bigger display, a more powerful chipset, more RAM in the base configuration and a tougher and more premium build.''',
	'''Most other aspects of what made the Galaxy A54 successful remain untouched - the microSD card support, good primary camera garnished with a decent ultrawide shooter, dependable battery life and IP67-rated build. Moreover, this time, it uses an aluminum frame and Gorilla Glass Victus+ protection.''',
	'''But will this be enough to persuade potential buyers? After all, the sub-€500 competition in Europe is fierce, and the same goes for the Indian INR 40,000-ish bracket. What made the A54 good in 2023 doesn't necessarily mean it will be enough in 2024. The competition is catching up, and we've seen some pretty solid midrangers come out in the first quarter of this year, so stick around to see if the A55 is worth your hard-earned money.''',
	'''The Galaxy A55 comes in a modest retail box with only some user manuals, a SIM tool and a USB-C to USB-C cable for charging and data transfer. There's no charger included.''',
	'''A few small but meaningful upgrades to the camera system, tweaks to the build and display, and the mandatory chipset refresh - we can't be mad at the Xiaomi 14 Ultra for its short list of changes, when last year's model was already one of the best cameraphones in existence.''',
	'''Indeed, the 14 Ultra doesn't stray too far from its predecessor. The main camera had a 1-inch sensor and dual-aperture lens on the 2023 model, the 2024 one uses a newer 1-inch sensor and makes that a truly variable aperture (with intermediate steps). The zoom cameras (two of them, as seemingly required to be called an Ultra), meanwhile, bring close focusing to the table and the longer of them also gets a slightly brighter lens.''',
	'''On the outside, Xiaomi has replaced the Corning-branded glass of the old model with an in-house concoction that should be more durable (asterisks, of course), and the <i>non-leather</i> leather has also been improved. There's a new aluminum alloy used for the frame in the International version, while China also gets a titanium option on top of that. Sounds like a win.''',
	'''Improvements in the display section are hard to spot, but there's a promise for slightly more brightness (hardly a gamechanger), while the new Snapdragon is the new Snapdragon - better than the old one, more or less the same across all 2024 mode''',
	'''Xiaomi has chosen not to mess around with battery capacity or charging speed on the global variant (Chinese version does get an extra 300mAh), and we think we could have used a bump in longevity - maybe we'll get that from elsewhere.''',
	'''Little has changed in the presentation and the 14 Ultra comes in the usual black thick cardboard box with simple Xiaomi lettering and a red Leica badge to add a splash of color. The internal packaging is all made of paper, so that's a win.''',
	'''The contents include a 90W charger (same as last year's) and a proprietary USB-A-to-C cable to go with it. Xiaomi has also bundled a translucent plastic snap-on back cover to protect the Ultra out of the box. It's on the cheap side, but something is better than nothing, plus the phone itself is supposedly more durable than the previous iteration. Let's see what that's about.''',
	'''Samsung has unveiled the new members of the Galaxy A series hot on the heels of the Galaxy S24 premiere. The Galaxy A35 and Galaxy A55 build on the positive reception of the previous models and bring updated designs, performance, and cameras. Today, we will explore the Galaxy A35 in detail.''',
	'''The new Samsung Galaxy A35 looks a lot like the Galaxy A34. As you can imagine, there are only a handful of upgrades, and they were not meant to tempt the Galaxy A34 owners to switch but more to lure new or existing Samsung users with much older phones.''',
	'''So, the Galaxy A35 starts with a new, updated dual-glass design and a more durable Gorilla Glass Victus+ front panel. The phone is IP67-rated for dust and water resistance and comes in a variety of catchy colors. The screen remains the same as on the Galaxy A34 - a 6.6-inch Super AMOLED of extended 1080p resolution and 120Hz refresh rate.''',
	'''The Galaxy A35 brings the same chipset we saw on the Galaxy A54 - the Exynos 1380 5G, so in a way, it can be a good alternative to the A54.''',
	'''The cameras haven't changed much since the Galaxy A34 - a high-res main camera, an 8MP ultrawide, a 5MP macro, and a 13MP selfie. There is a new primary, though, a 50MP OIS imager up from 48MP on the A34.''',
	'''Another new feature available on the Galaxy A35 is the support for Wi-Fi 6 networks.''',
	'''The Galaxy A34 is powered by a 5,000mAh with 25W fast charging support. It runs on Android 14 with One UI 6.1.''',
	'''The Galaxy A34 has a well-rounded specs sheet and no obvious omissions. However, it's very close to the Galaxy A54 feature set, so it remains to be seen if there is enough to set this new model apart. ''',
	'''The thin and lightweight cardboard retail box of the Galaxy A35 contains the phone and one basic passive 3A USB Type-C to Type-C cable. That's it. There are no plastics in sight, just cardboard.''',
	'''The Galaxy A35 supports 25W fast wired charging, but if you don't own a compatible USB-PD power adapter, you will need to shop for one, as there is no charger in the box.''',
	'''You look at the Galaxy S24 and S24+ and can't help but wonder if they really are the new models, or Samsung just went and reissued the old ones. The Ultra, on the other hand, is a different beast. The Galaxy S24 Ultra does make some significant strides towards improving on what was already the ultimate smartphone package.''',
	'''Some of the changes are immediately visible and start with the even blockier design with a fully flat face and the less reflective display glass. You won't immediately tell, but that glass should also be tougher, and as months go by, you'll perhaps appreciate the switch to titanium for the exposed frame, too.''',
	'''An even less obvious development, but one that has potential to be a major step in solidifying the Galaxy Ultra's position as a top cameraphone, is the new 5x zoom telephoto. Undoubtedly better at 5x than the old model, can it be as good at 10x?''',
	'''The Ultra won't partake in the chipset divide that's now returned to the lesser models, and all of the ultimate Galaxies will feature a Snapdragon 8 Gen 3 - no Exynos here if that makes any difference. Sure enough, the S24 Ultra will have all the AI smarts Samsung and Google came up with to fill an hour of Unpacked 2024.''',
	'''Unchanged is the battery capacity on this year's model, and so too is the maximum charging rating, for better or for worse. No change can be a good thing too, and the S Pen remains another differentiator that elevates the Galaxy Ultra above all others when it comes to productivity and just general usefulness (it's the best camera remote is what it is).''',
	'''You can have another look at the key specs below, before we proceed to get our hands on the Ultra, at long last.''',
	'''Another uneventful unboxing experience awaits, but it's not like the slimline box is making any promises. A SIM eject pin is still included, though a line in the instructions that says 'use a paper clip or an earring' could be replacing that in the near future. That, or eSIMs.''',
	'''The other thing of any practical use inside the box is a USB-C cable, but it's only good for 3A of current, and that may not be enough to get the promised 45W of charging. We might have a rant on that subject later on in the review.''',
	'''The Galaxy S Plus is probably the least popular device of the Galaxy S trio, but it has its fans. It caters to those who want a bigger Galaxy S screen and don't want to overpay for Ultra features they will not use. Today, we will be meeting with the Galaxy S24+, which finally returns to its former glory with a Quad HD display!''',
	'''Indeed, this has to be the biggest change since the Galaxy S21 series! The Galaxy S24+ brings back the 1440p OLED display we last saw on the Galaxy S20+ in 2020. And it's as premium of a panel as it can get - a 6.7-inch LTPO Dynamic AMOLED 2X screen with 513ppi, 120Hz dynamic refresh rate, HDR10+ and up to 2,600 nits of peak brightness.''',
	'''Quick note - we are reviewing the Exynos model of the Galaxy S24+ here.''',
	'''The refreshed design is the next thing on the list of upgrades - the aluminum frame is now tougher, flatter and all matte! This makes the Galaxy S24+ look new and feel new, too.''',
	'''Just like the Galaxy S24, the S24+ hardware varies depending on the market. The global models employ the Exynos 2400 chip with a 10-core CPU and new AMD graphics, while the US/Canada/China models run on the Snapdragon 8 Gen 3. The S24+ is available in 256GB and 512GB variants, both with UFS 4.0 chips and 12GB of RAM.''',
	'''The camera system is identical to the one on the Galaxy S24, S23+ and S23. There are three cameras on the back - a 50MP OIS primary, a 10MP 3x telephoto and a 12MP ultrawide. The front camera is a 12MP shooter with autofocus and 4K capturing.''',
	'''The Galaxy S24+ specs also include Ultra Wideband (UWB) support and a larger 4,900mAh battery with 45W wired and 15W wireless charging.''',
	'''The Galaxy S24+ seems like it has it all. Maybe it deserved a better zoom camera, or at least one of higher resolution, so it can allow for lossless/hybrid zoom. But we guess this incremental update will now be reserved for the Galaxy S25 next year.''',
	'''The Galaxy S24+ ships inside a thin black box containing a USB-C cable and a SIM ejection tool.<''',
	'''Those who pre-ordered the S24+ also get a 25W power adapter, a 15W charger pad, and a clear case. Those extras are market-dependent, of course.''',
	'''The cable that ships inside the retail boxes of the Galaxy S24 series is officially rated as 3A or up to 25W. This means if you buy a 45W charger or similar and it doesn't include a proper cable, you will need to shop for a compatible cable so you can enjoy faster than 25W charging.''',
]

In [51]:
gsm_arena_review = pd.DataFrame({'review': gsm_arena_review})
gsm_arena_review['review'] = gsm_arena_review['review'].apply(clean_text)

In [52]:
gsm_arena_review['tag'] = gsm_arena_review['review'].apply(tagging_data)

In [53]:
gsm_arena_review.to_csv('datasets/tagged/gsm_arena_review_datasets.csv', index=False)

In [54]:
review = pd.read_csv('datasets/tagged/reviews.csv', encoding='latin1')
cpu_name = pd.read_csv('datasets/tagged/cpu_name.csv', encoding='latin1')
more_review = pd.read_csv('datasets/tagged/gsm_arena_review_datasets.csv', encoding='latin1')

In [55]:
cpu_name.rename(columns={'cpu_name': 'review'}, inplace=True)

In [56]:
review.head()

,review,tag
0,gud product price,O O O
1,one best phone far price range,O O O O O O
2,mobail nice betrry bad fast charging lowest ca...,O O O O B-charging I-charging O B-camera O O O...
3,battery worrest,B-battery O
4,bought phone horrible experience redmi note 8 ...,O O O O O O O O O O O O O


In [57]:
all_data = pd.concat([review, cpu_name, more_review])
all_data = all_data.sample(frac=1)

In [58]:
to_iob_tag(all_data['review'], all_data['tag'], 'final/iob-ner-v3')

# **Training Model**

## **Prepare Data**

In [92]:
data = pd.read_csv("datasets/final/iob-ner-v3.csv",encoding="latin1")
data.head()

,sentence_id,words,labels
0,0,good,O
1,11111,2,O
2,22222,awesome,O
3,22222,phone,O
4,22222,need,O


In [93]:
X = data[["sentence_id","words"]]
Y = data["labels"]
label = ['O',
 'B-STORAGE',
 'B-PERFORMANCE',
 'B-BATTERY',
 'I-BATTERY',
 'B-CAMERA',
 'I-CAMERA',
 'B-CHARGING',
 'I-CHARGING',
 'I-STORAGE',
 'I-PERFORMANCE',
 'B-SCREEN',
 'I-SCREEN',
 'B-AUDIO',
 'B-FEATURE',
 'I-FEATURE',
 'B-CONNECTION',
 'I-CONNECTION',
 'B-BUTTON',
 'I-BUTTON',
 'I-AUDIO']

In [96]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
train_data = pd.DataFrame({"sentence_id": x_train["sentence_id"], "words": x_train["words"], "labels":y_train})
test_data = pd.DataFrame({"sentence_id": x_test["sentence_id"], "words": x_test["words"], "labels":y_test})

## **Training**

In [94]:
args = NERArgs()
args.num_train_epochs = 9
args.learning_rate = 1e-4
args.overwrite_output_dir =True
args.train_batch_size = 32
args.eval_batch_size = 32

model = NERModel('bert', 'dslim/bert-base-NER', labels=label, args=args, ignore_mismatched_sizes=True)

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at dslim/bert-base-NER and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([9]) in the checkpoint and torch.Size([21]) in the model instantiated
- classifier.weight: found shape torch.Size([9, 768])

In [ ]:
model.train_model(train_data, eval_data=test_data, acc=accuracy_score)

In [ ]:
result, model_outputs, preds_list = model.eval_model(test_data)
result

In [ ]:
with open('model/NER_Model_pickle_v4', 'wb') as file:
  pickle.dump(model, file)

# **Testing**

In [ ]:
with open('model/NER_Model_pickle_v4', 'rb') as file:
	model = pickle.load(file)

In [ ]:
txt = '''
Apple pulled off some unexpected surprises with the iPhone 14 Pro: there had been lots of solid rumors about the company switching from putting the front-facing camera and Face ID system in a pill-shaped cutout instead of the familiar notch, but the new “Dynamic Island” alert system came out of nowhere. And while it was getting clearer that Apple would have to follow the industry in using bigger camera sensors eventually, Apple went even further and rebooted its entire computational photography system as the Photonic Engine.
'''
txt = clean_text(txt)
print(txt)

In [ ]:
prediction, model_output = model.predict([txt])

In [ ]:
prediction

## **Re-train model using prediction -> still thinking**

In [ ]:
word = [key for item in prediction[0] for key in item.keys()]
word = ' '.join(word)

tag = [key for item in prediction[0] for key in item.values()]
tag = ' '.join(tag)